In [10]:
from thermoclassifier.elements.ensemble import EnsembleClassifierNet, majority_vote
from thermoclassifier.dataset.dataset_creator import DatasetCreator
import numpy as np
import torch
import torch.nn as nn

In [12]:
dc = DatasetCreator(elements=None,  splits=(0.8, 0.2), validation=False, seq_len=1, measurement='C', stable_only=True)
train_dataset, test_dataset, val_dataset = dc.get_datasets()

In [108]:
inp = torch.zeros([len(train_dataset), 2])
y = torch.zeros([len(train_dataset), ], dtype=torch.long)

idx = torch.randperm(len(train_dataset))

for d, i in zip(train_dataset, idx):
    inp[i] = torch.tensor(d[0, :2])
    y[i] = torch.tensor(d[0, -1])
    
inp = inp.unsqueeze(1)#[:40]
#y = y[:40]

In [ ]:
nets = []
optimizers = []
for i in range(10):
    nets.append(EnsembleClassifierNet(train=True, in_features=2))
    optimizers.append(torch.optim.Adam(nets[i].parameters(), lr=.001))

for i in range(2500000):
    #print('Epoch %i' %i)
    votes = torch.zeros(size=[len(nets), inp.shape[0]], dtype=torch.int)
    for i, (net, optimizer) in enumerate(zip(nets, optimizers)):
        out = net(inp)
        
        net.zero_grad()
        loss = nn.CrossEntropyLoss()(out, y)
        loss.backward()
        optimizer.step()

votes[i] = out.argmax(dim=-1)
l = votes.T.tolist()

In [ ]:
m = majority_vote(l, error_class=net.num_classes, threshold=0.0)
print('majority vote: ', m)
print('y:             ', y.tolist())

print((torch.tensor(m) == y).sum()/len(y))

In [ ]:
print(votes)